## 1. Hashing task!

* We've put all passwords from passwords1.txt and passwords2.txt  in two lists: listPasswords1 and listPasswords2

In [1]:
listPasswords1=[]
listPasswords2=[]
filePassword1 = open("passwords1.txt", 'r')
lines_p1 = filePassword1.readlines()
for line in lines_p1:
    listPasswords1.append(line.strip())

filePassword2 = open("passwords2.txt", 'r')
lines_p2 = filePassword2.readlines()
for line in lines_p2:
    listPasswords2.append(line.strip())



We have set all variables that we need to build our Bloom Filter:
* n = Number of items in the filter
* p = Probability of false positives, fraction between 0 and 1
* m = Number of bits in the filter (size of Bloom Filter bit-array)
* k = Number of hash functions

We did these following considerations:
N is the amount of passwords that are in passwordss1.txt (i.e. in listPasswords1).
We need to calculate p,m and k from following formulas:
\begin{align*}
k = (m/n)*ln(2)
\end{align*}

\begin{align*}
m = ((n*ln(p)) / (ln(2))^2
\end{align*}

Since we don't know the value of any previous varaibles, from https://hackernoon.com/probabilistic-data-structures-bloom-filter-5374112a7832 (suggested link in track) we have set initially p=0.0015. P must be a valuue between 0 and 1. With smaller value of p, we have a lower probability to have a false positives in search on bloom filter. 
Bloom filter can have false positives and NOT false negatives. We have choosen this value (0.0015) from a different test that we have tryed. We think that it should be an optimal value to have also optimal values about k and m.
Funally we calculated k and m (with previous formulas).

In [2]:
n=len(listPasswords1)
p=0.0015
import math
m=math.ceil((n * math.log(p)) / math.log(1 / pow(2, math.log(2))))
k = round((m / n) * math.log(2))

This following function is our hash function. We have written from scratch our fnv function based on fnv hash function. FNV hashes are designed to be fast while maintaining a low collision rate. The FNV speed allows one to quickly hash lots of data while maintaining a reasonable collision rate. 
To build bloom filter we need of various hash functions that should be independent, uniformly distributed and fast.
From previous formulas, we have calculated the number of different hash functions that we should use. We set a variable, called 'seed' that is an integer from 0 to k(number of hash functions); in this way we'll have k different hash fucntions that transform our input string in k different number. These k numbers will be the index of our bit-array that represent respective inout password.


In [4]:
def fnv1_64(string, seed=0):
    """
    Returns: The FNV-1 hash of a given string. 
    """
    #Constants
    FNV_prime = 1099511628211
    offset_basis = 14695981039346656037

    #FNV-1a Hash Function
    hash = offset_basis + seed
    for char in string:
        hash = hash * FNV_prime
        hash = hash ^ ord(char)
    return hash




This following class 'BloomFilter' represent out Bloom Filter. 

It has three attributes:
* sizeArray = dimension of bit-array
* number_HashFucntion = number of hash functions
* array_BloomFilter = bit-array of bloom filter

It has also two methods:
* init: it takes, as parameteers ouur bloom filter, k (number of hash functions that we have calculated from previous formula), m  (size of bit-array that we have calculated with  previous function). It initializes the bit-array of Bloom filter (size=m) with all '0'; its size with m and its number of hash functions with k.
* add: this method allow to add an element from input list to our bloom filter. It takes this list and our bloom filter as parameters and it calculates the effective bit-array with right '1' and '0'.

In [5]:
class BloomFilter:

    sizeArray=0
    number_HashFucntion=0
    array_BloomFilter=[]
    count=0
    @property
    def size(self):
        return self.sizeArray
    @property
    def numHash(self):
        return self.number_HashFucntion
    @property
    def arrayBloom(self):
        return self.array_BloomFilter
    
    def init(self,k,m):
        self.sizeArray=m
        self.number_HashFucntion=k
        
        for i in range(n):
            self.array_BloomFilter.append(0)
    
    def add(self,strings):
        print(self.number_HashFucntion)
        print(self.sizeArray)
        h=0
        for psw in strings:
            a=false
            for seed in range(self.number_HashFucntion):
                index=fnv1_64(psw,seed) % self.sizeArray
                if self.array_BloomFilter[index]==1 and a==false:
                    self.count+=1
                    a=true
                self.array_BloomFilter[index]=1

Then we made a function 'checkPassw' that checks how many password in listPasswords2 (i.e. in passowrds2.txt) are in our Bloom Filter. It takes our bloom filter and the list of passwords that are in passwords2.txt and it returns how many passwords are in bloom filter. 
A password is in our bloom filter if and only if its conversion with hash functions coresponds to every '1' in the bit-array. If there is only one '0', current password is not in bloom filter.

'countCheck' is the number of occurences of passwords (from passwords2.txt) in bloom filter. This number represent the occurences of passwords from passwords2.txt that are 'probably' in password1.txt

In [42]:
def checkPassw(BloomFilter, listPasswords2):
    countCheck=0
    for psw in listPasswords2:
        count=0
        for seed in range(BloomFilter.number_HashFucntion):
            index=fnv1_64(psw,seed) % BloomFilter.sizeArray
            if BloomFilter.array_BloomFilter[index]==1:
                count+=1
            if count==BloomFilter.number_HashFucntion:
                countCheck+=1
            
    return countCheck

## Bonus section: 
###### We calculate the number of false positive

This following function 'falsePositives' allows to calculate the exact number of false positive. It takes these following data, as parameter:
* Bloom Filter = our bloom filter
* listPassowrds1 = passwords from passwords1.txt
* listPassowrds2 = passwords from passwords2.txt

For every password in listPasswords2, it checks for every password, if current password is in the bloom filter. Then if this password is in bloom filter, it means that it is 'probably' in listPasswords2. To check if it's a false positive (i.e. it is not in listPAsswords1), we'll continue to verify if this current password is in listPassword1. If it's true, it means tht it is a False psiitive and we increment our counter 'countFalsePositives'; if it's True it means this password is actually in listPassword1 nad we continue with next password in listPassword2.

In [44]:
def falsePositives(BloomFilter, listPasswords1, listPasswords2):
    countFalsePositives=0
    for psw in listPasswords2:
        count=0
        for seed in range(BloomFilter.number_HashFucntion):
            index=fnv1_64(psw,seed) % BloomFilter.sizeArray
            if (BloomFilter.array_BloomFilter[index]==1):
                    count+=1
            else:
                break
            if count==BloomFilter.number_HashFucntion and not(psw in listPasswords1):
                countFalsePositives+=1
                
    return countFalsePositives
                

Then we did the main function, as wirtten in homework track. With this function we did these following steps:
* init bit-array, its size adn number of hash functions of our bloom filter, with 'BloomFilter.init(BloomFilter,k,m)'
* add passowrds from listPasswords1 into our bloom filter, with 'BloomFilter.add(BloomFilter,listPasswords1)'
* calculate how many passwords (from passwords2.txt) are present in our bloom filter, with 'checkPassw(BloomFilter,listPasswords2)'

###### Finally we print these following functions:
* Number of hash function used
* Number of duplicates detected
* Probability of false positives
* Execution time

In [6]:
import time

def BloomFilterFunc(listPasswords1, listPasswords2):
    start = time.time()
    #init our bloom filter
    BloomFilter.init(BloomFilter,k,m)
    #add all passowrd from listPassowrds1 to our bloom filter
    BloomFilter.add(BloomFilter,listPasswords1)
    #check and save into 'countPassw' the number of occurences of password (from passwords2) in the bloom filter
    countPassw=checkPassw(BloomFilter,listPasswords2)
    end = time.time()
    
    #print output data
    
    print('Number of occurences of password (from passwords2) in the bloom filter: ', countPassw)
    print('Number of hash function used: ', k)
    print('Number of duplicates detected: ', BloomFilter.count)
    print('Probability of false positives: ', p)
    print('Execution time: ', end-start)
    



* Execute main function

In [ ]:
BloomFilterFunc(listPasswords1, listPasswords2)

* Execute bonus section

In [ ]:
print('Number of flase positive: ', falsePositives(BloomFilter,listPasswords1,listPasswords2))